<a href="https://colab.research.google.com/github/ilquerre/SpecialTopicsinRemoteSensing/blob/main/AdvancedRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=evNfroH69J85h0AGBr1tI05qPAnCZ2wAKSNdsrdFL7s&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6wmMBLt_HWInhm99zJ5VOFdaa2P25JGiJEfDE4x2_App4zW_Yok00

Successfully saved authorization token.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

In [ ]:

geoJSON={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              41.407470703125,
              37.90953361677018
            ],
            [
              41.43013000488281,
              37.640334898059486
            ],
            [
              41.67869567871094,
              37.64196606132065
            ],
            [
              41.66496276855469,
              37.90628303986326
            ],
            [
              41.407470703125,
              37.90953361677018
            ]
          ]
        ]
      }
    }
  ]
}

In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)
ls=ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(aoi)
st=ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(aoi)

In [ ]:
lsimgold=ee.Image(ls.filterDate(ee.Date("2017-07-01"),ee.Date("2017-12-31")).sort('CLOUD_COVER').first().clip(aoi))
lsimgnew=ee.Image(ls.filterDate(ee.Date("2021-04-01"),ee.Date("2021-12-31")).sort('CLOUD_COVER').first().clip(aoi))

stimgold=ee.Image(st.filterDate(ee.Date("2017-07-01"),ee.Date("2017-12-31")).sort('CLOUD_COVERAGE_ASSESSMENT').first().clip(aoi))
stimgnew=ee.Image(st.filterDate(ee.Date("2021-04-01"),ee.Date("2021-12-31")).sort('CLOUD_COVERAGE_ASSESSMENT').first()).clip(aoi)



In [ ]:
url = lsimgold.select(['B4',"B3","B2"]).getThumbURL({"min":0,"max":10000})
disp.Image(url=url, width=400)


In [ ]:
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
location = aoi.centroid().coordinates().getInfo()[::-1]
visParams = {"bands": ['B4', 'B3', 'B2'],
  "min": 0,
  "max": 3000,
  "gamma": 1.4}
# Make an RGB color composite image (VV,VH,VV/VH).
sentold = ee.Image(stimgold)
landold = ee.Image(lsimgold)
sentnew = ee.Image(stimgnew)
landnew = ee.Image(lsimgnew)

# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(sentold, visParams, 'Sentinel-OLD')
m.add_ee_layer(landold, visParams, 'Landsat-OLD')
m.add_ee_layer(sentnew, visParams, 'Sentinel-NEW')
m.add_ee_layer(landnew, visParams, 'Landsat-NEW')



# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [ ]:
stimgnew.getInfo()

{'bands': [{'crs': 'EPSG:32637',
   'crs_transform': [60, 0, 699960, 0, -60, 4200000],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [420, 522],
   'id': 'B1',
   'origin': [186, 0]},
  {'crs': 'EPSG:32637',
   'crs_transform': [10, 0, 699960, 0, -10, 4200000],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [2512, 3132],
   'id': 'B2',
   'origin': [1121, 0]},
  {'crs': 'EPSG:32637',
   'crs_transform': [10, 0, 699960, 0, -10, 4200000],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [2512, 3132],
   'id': 'B3',
   'origin': [1121, 0]},
  {'crs': 'EPSG:32637',
   'crs_transform': [10, 0, 699960, 0, -10, 4200000],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [2512, 3132],
   'id': 'B4',
   'origin': [1121, 0]},
  {'crs': 'EPSG:32

In [ ]:
lsimgold.getInfo()

{'bands': [{'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 623985, 0, -30, 4266015],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [838, 1139],
   'id': 'B1',
   'origin': [2906, 2106]},
  {'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 623985, 0, -30, 4266015],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [838, 1139],
   'id': 'B2',
   'origin': [2906, 2106]},
  {'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 623985, 0, -30, 4266015],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [838, 1139],
   'id': 'B3',
   'origin': [2906, 2106]},
  {'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 623985, 0, -30, 4266015],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [838, 1139],
   'id': 'B4',
   'origin': [2